# setup

In [5]:
import torch
import sys
sys.path.append("/home/yandex/MLW2022/idoc/detectron2/detectron2/")
import detectron2
from detectron2.modeling import ROI_HEADS_REGISTRY
from detectron2.modeling.roi_heads import Res5ROIHeads, select_foreground_proposals
from detectron2.utils.logger import setup_logger
l = setup_logger(output="./my_log_file.log")
import matplotlib.pyplot as plt
import numpy as np
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator

from detectron2.data.datasets import register_coco_instances

from utils import MyTrainer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from pycocotools.cocoeval import COCOeval


In [6]:
class My_COCOeval(COCOeval):
    def summarize(self):
        '''
        Compute and display summary metrics for evaluation results.
        Note this functin can *only* be applied on the default parameter setting
        '''
        def _summarize( ap=1, iouThr=None, areaRng='all', maxDets=100 ):
            p = self.params
            iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
            titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
            typeStr = '(AP)' if ap==1 else '(AR)'
            iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
                if iouThr is None else '{:0.2f}'.format(iouThr)

            aind = [i for i, aRng in enumerate(p.areaRngLbl) if aRng == areaRng]
            mind = [i for i, mDet in enumerate(p.maxDets) if mDet == maxDets]
            if ap == 1:
                # dimension of precision: [TxRxKxAxM]
                s = self.eval['precision']
                # IoU
                if iouThr is not None:
                    t = np.where(iouThr == p.iouThrs)[0]
                    s = s[t]
                s = s[:,:,:,aind,mind]
            else:
                # dimension of recall: [TxKxAxM]
                s = self.eval['recall']
                if iouThr is not None:
                    t = np.where(iouThr == p.iouThrs)[0]
                    s = s[t]
                s = s[:,:,aind,mind]
            if len(s[s>-1])==0:
                mean_s = -1
            else:
                mean_s = np.mean(s[s>-1])

                #cacluate AP(average precision) for each category
                num_classes = 80
                avg_ap = 0.0
                if ap == 1:
                    for i in range(0, num_classes):
                        print('category : {0} : {1}'.format(i,np.mean(s[:,:,i,:])))
                        avg_ap +=np.mean(s[:,:,i,:])
                    print('(all categories) mAP : {}'.format(avg_ap / num_classes))

            print(iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s))
            return mean_s
        def _summarizeDets():
            stats = np.zeros((12,))
            stats[0] = _summarize(1)
            stats[1] = _summarize(1, iouThr=.5, maxDets=self.params.maxDets[2])
            stats[2] = _summarize(1, iouThr=.75, maxDets=self.params.maxDets[2])
            stats[3] = _summarize(1, areaRng='small', maxDets=self.params.maxDets[2])
            stats[4] = _summarize(1, areaRng='medium', maxDets=self.params.maxDets[2])
            stats[5] = _summarize(1, areaRng='large', maxDets=self.params.maxDets[2])
            stats[6] = _summarize(0, maxDets=self.params.maxDets[0])
            stats[7] = _summarize(0, maxDets=self.params.maxDets[1])
            stats[8] = _summarize(0, maxDets=self.params.maxDets[2])
            stats[9] = _summarize(0, areaRng='small', maxDets=self.params.maxDets[2])
            stats[10] = _summarize(0, areaRng='medium', maxDets=self.params.maxDets[2])
            stats[11] = _summarize(0, areaRng='large', maxDets=self.params.maxDets[2])
            return stats
        def _summarizeKps():
            stats = np.zeros((10,))
            stats[0] = _summarize(1, maxDets=20)
            stats[1] = _summarize(1, maxDets=20, iouThr=.5)
            stats[2] = _summarize(1, maxDets=20, iouThr=.75)
            stats[3] = _summarize(1, maxDets=20, areaRng='medium')
            stats[4] = _summarize(1, maxDets=20, areaRng='large')
            stats[5] = _summarize(0, maxDets=20)
            stats[6] = _summarize(0, maxDets=20, iouThr=.5)
            stats[7] = _summarize(0, maxDets=20, iouThr=.75)
            stats[8] = _summarize(0, maxDets=20, areaRng='medium')
            stats[9] = _summarize(0, maxDets=20, areaRng='large')
            return stats
        if not self.eval:
            raise Exception('Please run accumulate() first')
        iouType = self.params.iouType
        if iouType == 'segm' or iouType == 'bbox':
            summarize = _summarizeDets
        elif iouType == 'keypoints':
            summarize = _summarizeKps
        self.stats = summarize()

In [12]:
class My_COCOEvaluator(COCOEvaluator):
    def _evaluate_predictions_on_coco(
        coco_gt, coco_results, iou_type, kpt_oks_sigmas=None, use_fast_impl=False, img_ids=None
    ):
        """
        Evaluate the coco results using COCOEval API.
        """
        assert len(coco_results) > 0

        if iou_type == "segm":
            coco_results = copy.deepcopy(coco_results)
            # When evaluating mask AP, if the results contain bbox, cocoapi will
            # use the box area as the area of the instance, instead of the mask area.
            # This leads to a different definition of small/medium/large.
            # We remove the bbox field to let mask AP use mask area.
            for c in coco_results:
                c.pop("bbox", None)

        coco_dt = coco_gt.loadRes(coco_results)
        coco_eval = (COCOeval_opt if use_fast_impl else My_COCOeval)(coco_gt, coco_dt, iou_type)
        if img_ids is not None:
            coco_eval.params.imgIds = img_ids

        if iou_type == "keypoints":
            # Use the COCO default keypoint OKS sigmas unless overrides are specified
            if kpt_oks_sigmas:
                assert hasattr(coco_eval.params, "kpt_oks_sigmas"), "pycocotools is too old!"
                coco_eval.params.kpt_oks_sigmas = np.array(kpt_oks_sigmas)
            # COCOAPI requires every detection and every gt to have keypoints, so
            # we just take the first entry from both
            num_keypoints_dt = len(coco_results[0]["keypoints"]) // 3
            num_keypoints_gt = len(next(iter(coco_gt.anns.values()))["keypoints"]) // 3
            num_keypoints_oks = len(coco_eval.params.kpt_oks_sigmas)
            assert num_keypoints_oks == num_keypoints_dt == num_keypoints_gt, (
                f"[COCOEvaluator] Prediction contain {num_keypoints_dt} keypoints. "
                f"Ground truth contains {num_keypoints_gt} keypoints. "
                f"The length of cfg.TEST.KEYPOINT_OKS_SIGMAS is {num_keypoints_oks}. "
                "They have to agree with each other. For meaning of OKS, please refer to "
                "http://cocodataset.org/#keypoints-eval."
            )

        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

        return coco_eval

In [8]:
register_coco_instances("ww_train", {}, "../data/ww_train.json", "../data/")
register_coco_instances("ww_eval", {}, "../data/ww_val.json", "../data/")
register_coco_instances("ww_test", {}, "../data/ww_test.json", "../data/")

AssertionError: Dataset 'ww_train' is already registered!

In [9]:
@ROI_HEADS_REGISTRY.register()
class My_Res5ROIHeads(Res5ROIHeads):
    # def __init__(
    #         self,
    #         *,
    #         in_features: List[str],
    #         pooler: ROIPooler,
    #         res5: nn.Module,
    #         box_predictor: nn.Module,
    #         mask_head: Optional[nn.Module] = None,
    #         **kwargs,
    # ):
    #     super().__init__(in_features,
    #                      pooler,
    #                      res5,
    #                      box_predictor,
    #                      mask_head, kwargs)
    def forward(
            self,
            images,
            features,
            proposals,
            targets=None,
    ):
        """
        See :meth:`ROIHeads.forward`.
        """
        del images

        if self.training:
            assert targets
            proposals = self.label_and_sample_proposals(proposals, targets)
        del targets

        proposal_boxes = [x.proposal_boxes for x in proposals]
        box_features = self._shared_roi_transform(
            [features[f] for f in self.in_features], proposal_boxes
        )
        predictions = self.box_predictor(box_features.mean(dim=[2, 3]))

        if self.training:
            del features
            losses = self.box_predictor.losses(predictions, proposals)
            if self.mask_on:
                proposals, fg_selection_masks = select_foreground_proposals(
                    proposals, self.num_classes
                )
                # Since the ROI feature transform is shared between boxes and masks,
                # we don't need to recompute features. The mask loss is only defined
                # on foreground proposals, so we need to select out the foreground
                # features.
                mask_features = box_features[torch.cat(fg_selection_masks, dim=0)]
                del box_features
                losses.update(self.mask_head(mask_features, proposals))
            return [], losses
        else:
            pred_instances, _ = self.box_predictor.inference(predictions, proposals)
            pred_instances = self.forward_with_given_boxes(features, pred_instances)
        # for editing the proposals boxes  proposals[0]._fields["proposal_boxes"].tensor
        orig_res = pred_instances, {}
        # orig_res = super().forward(images, features, proposals, targets)
        if not self.training:
            # print("=" * 9 + "my module" + "=" * 9)
            if self.filter_flag:
                ins = pred_instances[0]
                # ins[ins._fields["pred_classes"] == 2]._fields["pred_boxes"]
                keep = torch.ones(len(ins), dtype=bool,device=ins._fields["pred_classes"].device)
                iou_mat = detectron2.structures.pairwise_iou(
                    ins[ins._fields["pred_classes"] == self.OPEN_FLOCK]._fields["pred_boxes"],
                    ins[ins._fields["pred_classes"] == self.SPHERICAL_FLOCK]._fields["pred_boxes"])
                keep[ins._fields["pred_classes"] == self.OPEN_FLOCK] = iou_mat.max(dim=1).values > self.filter_threshold
                ins._fields["pred_classes"] = ins._fields["pred_classes"][keep]
                ins._fields["scores"] = ins._fields["scores"][keep]
                ins._fields["pred_boxes"] = ins._fields["pred_boxes"][keep]
                # detectron2.structures.boxes.pairwise_intersection(
                #     ins[ins._fields["pred_classes"] == 2]._fields["pred_boxes"],
                #     ins[ins._fields["pred_classes"] == 2]._fields["pred_boxes"])
            # get the predicted boxes with pred_instances[0]._fields["pred_boxes"].tensor
            # get the class predicted for that box with pred_instances[0]._fields["pred_classes"]
            # todo need to make sure that removing a sample from the prediction don't break anything

            # print(orig_res)
            # print("=" * 22)
        return orig_res

# config

In [10]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"))
cfg.DATASETS.TRAIN = ("ww_train",)
cfg.DATASETS.TEST = ("ww_eval",)  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 1000  # 300 iterations seems good enough, but you can certainly train longer
# cfg.MODEL.ROI_HEADS.NAME = "My_Res5ROIHeads"
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # 3 classes (data, fig, hazelnut)
cfg.TEST.EVAL_PERIOD = 100

# Train

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.model.roi_heads.filter_threshold = 0.13
trainer.model.roi_heads.SPHERICAL_FLOCK = 4
trainer.model.roi_heads.OPEN_FLOCK = 2
trainer.model.roi_heads.filter_flag = False
trainer.train()

# Eval

In [14]:
model = build_model(cfg)
DetectionCheckpointer(model).load("output/model_final.pth")
test_loader = build_detection_test_loader(cfg, "ww_test")
evaluator = My_COCOEvaluator("ww_test", cfg, False, output_dir="../output1/")
out = inference_on_dataset(model, test_loader, evaluator)

WARNING [08/13 12:47:26 d2.modeling.backbone.resnet]: ResNet.make_stage(first_stride=) is deprecated!  Use 'stride_per_block' or 'stride' instead.
WARNING [08/13 12:47:27 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/13 12:47:27 d2.data.datasets.coco]: Loaded 264 images in COCO format from ../data/ww_test.json
[08/13 12:47:27 d2.data.common]: Serializing 264 elements to byte tensors and concatenating them all ...
[08/13 12:47:27 d2.data.common]: Serialized dataset takes 0.06 MiB
[08/13 12:47:27 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/13 12:47:27 d2.evaluation.evaluator]: Start inference on 264 images
[08/13 12:47:28 d2.evaluation.evaluator]: Inference done 11/264. 0.0879 s / img. ETA=0:00:29
[08/13 12:47:33 d2.evaluation.evaluator]: Inference done 54/264. 0.0910 s / img. ETA=0:00:24
[08/13 12:47:38 d2.evalu

In [12]:
out

OrderedDict([('bbox',
              {'AP': 8.259838370008726,
               'AP50': 24.228138871452074,
               'AP75': 3.5146747825044558,
               'APs': nan,
               'APm': nan,
               'APl': 8.30319171057265,
               'AP-Micro': nan,
               'AP-filament': 0.5049504950495048,
               'AP-open_floc': 6.6306046206946485,
               'AP-protozoa': 6.849856032187854,
               'AP-spherical_floc': 19.0539423321029})])